# Evaluate Division

You are given an array of variable pairs `equations` and an array of positive real numbers `values`, where <code>equations[i] = [A<sub>i</sub>, B<sub>i</sub>]</code> and `values[i]` represent the equation <code>A<sub>i</sub>/B<sub>i</sub>= values[i]</code>. Each <code>A<sub>i</sub></code> or <code>B<sub>i</sub></code> is a string that represents a single variable.

You are also given some `queries`, where <code>queries[j] = [C<sub>j</sub>, D<sub>j</sub>]</code> represents the <code>j<sup>th</sup></code> query where you must find the answer for <code>A<sub>j</sub></code> or <code>D<sub>j</sub></code> = ?.

Return the answers to all queries. If a single answer cannot be determined, return `-1.0`.

**Note:** The input is always valid. You may assume that evaluating the queries will not result in division by zero and that there is no contradiction.

**Note:** The variables that do not occur in the list of equations are undefined, so the answer cannot be determined for them.

# Examples

**Example 1:**
```Python
Input: 
    equations = [["a","b"],["b","c"]], 
    values = [2.0,3.0], 
    queries = [["a","c"],["b","a"],["a","e"],["a","a"],["x","x"]]
Output: [6.00000,0.50000,-1.00000,1.00000,-1.00000]
```
Explanation: 
- Given: a / b = 2.0, b / c = 3.0
- queries are: a / c = ?, b / a = ?, a / e = ?, a / a = ?, x / x = ? 
- return: [6.0, 0.5, -1.0, 1.0, -1.0 ]

Note: x is undefined => -1.0


**Example 2:**

```Python
Input: 
    equations = [["a","b"],["b","c"],["bc","cd"]],
    values = [1.5,2.5,5.0],
    queries = [["a","c"],["c","b"],["bc","cd"],["cd","bc"]]
Output: [3.75000,0.40000,5.00000,0.20000]
```

**Example 3:**
```Python
Input: 
    equations = [["a","b"]], 
    values = [0.5], 
    queries = [["a","b"],["b","a"],["a","c"],["x","y"]]
Output: [0.50000,2.00000,-1.00000,-1.00000]
```

# Analysis
We construct a graph where the nodes are all the variables appearing in the `equations`, and there is an edge between two nodes (variables) `a` and `b` if and only if `[a, b]` or `[b, a]` is in `equations`.

Then for any query `[a,b]`,
- if one of them is not in the graph, the answer is `-1.0`;
- if both of them are in the graph and `a == b`, the answer is `1.0`;
- if both of them are in the graph and `a != b`, then the answer exists if and only if there exists a path <code>a_1, a_2, ..., a_n</code> such that `a = a_1`, `b = a_n`. Indeed, in this case, each <code>a<sub>i</sub>/a<sub>i+1</sub></code> is known and the answer is 
<div align="center">

```
a/b=a_1/a_2 * a_2/a_3 * ... * a_{n-1}/a_n
```

</div>

In [ ]:
from collections import defaultdict, deque
import math


class Solution:
    def calcEquation(
        self, equations: list[list[str]], values: list[float], queries: list[list[str]]
    ) -> list[float]:
        variables = defaultdict(dict)
        for i, (a, b) in enumerate(equations):
            variables[a][b] = values[i]
            variables[b][a] = 1 / values[i]

        def get_path(a, b):
            queue = deque([(a, [a])])
            visited = set()
            while len(queue) > 0:
                node, path = queue.popleft()
                if node in visited:
                    continue
                if node == b:
                    return path
                visited.add(node)

                for neighbor in variables[node]:
                    if neighbor not in visited:
                        queue.append((neighbor, path + [neighbor]))
            return None

        def calculate_quotient(a, b):
            if a not in variables or b not in variables:
                return -1.0
            if a == b:
                return 1.0
            path = get_path(a, b)
            if path == None:
                return -1.0
            return math.prod(
                [variables[path[i]][path[i + 1]] for i in range(len(path) - 1)]
            )

        return [calculate_quotient(a, b) for a, b in queries]